In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="gpt-oss:20b"
)

d:\Learning-lab\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.agents import create_agent

def get_weather(city: str)->str:
    """回傳關於城市的天氣報告"""
    print(f"目標城市： {city} 的天氣報告")
    return "天氣晴朗，氣溫30度。"

agent = create_agent(
    model= llm,# 可傳入：模型名/模型物件，但是不能傳入chain/pipeline
    tools=[get_weather],
    system_prompt="你是一位有禮貌的助理，你只能使用繁體中文回答問題。"
)

In [14]:
result = agent.invoke(
    {
        "messages":[
            {"role":"user","content":"台南的天氣如何?"}
            ],
    }
)

print(result)

目標城市： 台南 的天氣報告
{'messages': [HumanMessage(content='台南的天氣如何?', additional_kwargs={}, response_metadata={}, id='fd6d4011-476f-4ba7-a7d4-55d0bf9aa78c'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-11-03T08:45:38.4137338Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5455251700, 'load_duration': 199281200, 'prompt_eval_count': 155, 'prompt_eval_duration': 1407667100, 'eval_count': 53, 'eval_duration': 3780832800, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--c72f4b20-a8dd-4e01-b5e4-ef0782c7c1ff-0', tool_calls=[{'name': 'get_weather', 'args': {'city': '台南'}, 'id': '2773b565-fff9-497a-9481-b17df0ab11f2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 155, 'output_tokens': 53, 'total_tokens': 208}), ToolMessage(content='天氣晴朗，氣溫30度。', name='get_weather', id='49e08566-54f8-40d8-ba03-4bfafbeef4f4', tool_call_id='2773b565-fff9-497a-9481-b17df0ab11f2'), AIMessage(content='您好！根據最新天氣查詢，台南今天天氣晴朗，

In [3]:
from langchain.messages import AIMessage, ToolMessage, HumanMessage
def get_message(result):
    for msg in result["messages"]:
        if isinstance(msg, ToolMessage):
            print("Tool output:", msg.content)
        elif isinstance(msg, AIMessage):
            print("AI output:", msg.content)
        elif isinstance(msg, HumanMessage):
            print("Human input:", msg.content)

In [9]:
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from dataclasses import dataclass

@dataclass
class Context:
    user_id: str
    session_id: str

agent2 = create_agent(
    model=llm,
    tools=[get_weather],
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "get_weather":{
                    "allowed_decisions":["approve","edit","reject"]
                }
            }
        )
    ],
    checkpointer=InMemorySaver(),
    context_schema=Context
)
config = {"configurable":{"thread_id":"123"}}

In [10]:
result = agent2.invoke(
    {
        "messages":[
            {"role":"user","content":"台北的天氣如何?"}
            ],
    },
    context=Context(user_id="123",session_id="123"),
    config=config
)
if "__interrupt__" in result:
    interrupt = result["__interrupt__"][0]
    print("等待人工批准：", interrupt.value)

等待人工批准： {'action_requests': [{'name': 'get_weather', 'args': {'city': '台北'}, 'description': "Tool execution requires approval\n\nTool: get_weather\nArgs: {'city': '台北'}"}], 'review_configs': [{'action_name': 'get_weather', 'allowed_decisions': ['approve', 'edit', 'reject']}]}


In [ ]:
# 接受工具請求
from langgraph.types import Command

result = agent2.invoke(
    Command(
        resume={
            "decisions": [
                {
                    "type":"approve",
                }
            ]
        }
    ),
    context=Context(user_id="123",session_id="123"),
    config=config
)
get_message(result)

目標城市： 台北 的天氣報告
Human input: 台北的天氣如何?
AI output: 
Tool output: 天氣晴朗，氣溫30度。
AI output: 台北今天天氣晴朗，氣溫大約 30°C。白天陽光充足，傍晚可稍微降溫幾度，風速約 5‑10 公里／小時。若您有其他天氣需求（例如風向、濕度、空氣品質、未來天氣預報等），請隨時告訴我！


In [ ]:
# 修改工具請求
from langgraph.types import Command

result = agent2.invoke(
    Command(
        resume={
            "decisions": [
                {
                    "type":"edit",
                    "edited_action": {                                 
                    "name": "get_weather",
                    "args": {"city": "高雄"} 
                    }
                }
            ]
        }
    ),
    context=Context(user_id="123",session_id="123"),
    config=config
)
get_message(result)

# edit tool參數，tool回覆變更後，會與使用者原始提問不同
# 這種上下文情況沒有修改prompt的話，會影響model回覆。
# 此處model 回覆空字串。
# 或許可以透過動態prompt方法解決該問題。

目標城市： 高雄 的天氣報告
Human input: 台北的天氣如何?
AI output: 
Tool output: 天氣晴朗，氣溫30度。
AI output: 


In [ ]:
# 拒絕工具請求
from langgraph.types import Command

result = agent2.invoke(
    Command(
        resume={
            "decisions": [
                {
                    "type":"reject",
                }
            ]
        }
    ),
    context=Context(user_id="123",session_id="123"),
    config=config
)
get_message(result)

Human input: 台南的天氣如何?
AI output: 
Tool output: User rejected the tool call for `get_weather` with id ad46b920-d2a5-45e9-b327-057bd95b5eaf
AI output: 抱歉，我目前無法即時取得最新天氣資訊。一般來說，台南屬於亞熱帶氣候，春季多為晴朗且氣溫在20–28 °C之間；夏季則熱且濕度高，偶爾會有短暫雷雨。若您想知道今天或未來幾天的實際天氣狀況，建議查看官方氣象局網站、台灣天氣APP，或在網路搜尋「台南天氣預報」即可獲得即時更新。若您需要更多天氣相關建議（例如出門注意事項、行程規畫等），也歡迎再告訴我！


In [ ]:
# 動態提示
"""
from dataclasses import dataclass

from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langgraph.runtime import Runtime


@dataclass
class Context:  
    user_role: str = "user"

@dynamic_prompt
def dynamic_prompt(request: ModelRequest) -> str:  
    user_role = request.runtime.context.user_role
    base_prompt = "You are a helpful assistant."

    if user_role == "expert":
        prompt = (
            f"{base_prompt} Provide detailed technical responses."
        )
    elif user_role == "beginner":
        prompt = (
            f"{base_prompt} Explain concepts simply and avoid jargon."
        )
    else:
        prompt = base_prompt

    return prompt  

agent = create_agent(
    model="gpt-4o",
    tools=tools,
    middleware=[dynamic_prompt],  
    context_schema=Context
)

# Use with context
agent.invoke(
    {"messages": [{"role": "user", "content": "Explain async programming"}]},
    context=Context(user_role="expert")
)
"""

In [ ]:
# 呼叫前模型hook
# 可用於：總結對話歷史 / 修剪訊息長度 / 輸入保護（敏感資訊處理）
"""
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware

agent = create_agent(
    model="claude-sonnet-4-5-20250929",
    tools=tools,
    middleware=[
        SummarizationMiddleware(  
            model="claude-sonnet-4-5-20250929",  
            max_tokens_before_summary=1000
        )  
    ]  
)"""

In [ ]:
# 呼叫後模型hook
# 可用於：人類審核 / 輸出保護
"""from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware

agent = create_agent(
    model="claude-sonnet-4-5-20250929",
    tools=[read_email, send_email],
    middleware=[HumanInTheLoopMiddleware(
        interrupt_on={
            "send_email": True,
            "description": "Please review this email before sending"
        },
    )]
)"""